In [1]:
from python_graphql_client import GraphqlClient
import pandas as pd
from datetime import datetime
import matplotlib

In [7]:
def client():
    return GraphqlClient(endpoint="https://api.thegraph.com/subgraphs/name/token-terminal-subgraphs/tornado-cash-v1-polygon")

def query(event, skip, timestamp):
    return """
    {
      """+event+"""(first: 1000, orderBy: blockTimestamp, skip: """+ str(skip)+""",
      where: {blockTimestamp_gt: """+ str(timestamp) +"""}) {
        id
        blockTimestamp
      }

    }
"""

In [11]:
def getPoolEvents(event, debug=False):
    timestamp=0
    poolEvents=0
    
    while(True):
        skip=0
        poolEventsTs=0
        while(skip <= 5000):
            querys = query(event, skip, timestamp)
            try:
                tgdata_raw = client().execute(query=querys)['data'][event]
                if debug : print(skip, timestamp, len(tgdata_raw))
            except:
                print(client().execute(query=querys))
                break
            tgdata = pd.DataFrame.from_dict(tgdata_raw)
            if type(poolEventsTs) is int : poolEventsTs = tgdata 
            else : poolEventsTs = pd.concat([poolEventsTs,tgdata], ignore_index=True)

            if tgdata.shape[0] < 1000: break
            skip = skip + 1000
        if type(poolEvents) is int :  poolEvents = poolEventsTs 
        else :  poolEvents = pd.concat([poolEvents,poolEventsTs], ignore_index=True)

        if poolEventsTs.shape[0] < 6000: break
        timestamp = str(int(poolEventsTs['blockTimestamp'].iloc[-1]) -1)

    return poolEvents

poolEventsList =[]
for size in ['100','1000','10000','100000']:
    deposits = getPoolEvents(f'matic{size}V1DepositV1Es').drop_duplicates()
    deposits['type'] = 'D'
    withdrawls = getPoolEvents(f'matic{size}V1WithdrawalV1Es').drop_duplicates()
    withdrawls['type'] = 'W'
    combined = pd.concat([deposits, withdrawls])
    combined['amount'] = size
    combined['currency'] = 'matic'
    poolEventsList.append(combined)

poolEvents = pd.concat(poolEventsArray)

In [22]:
poolEvents

,id,blockTimestamp,type,amount,currency
0,0x6e08d0ecee7067f1dfb7cc581ac6cb71c38fc9c111c3...,1624912698,D,100,matic
1,0xa19d8be5f89962f72d932568e4c39d2d34f985e5eaf8...,1624913658,D,100,matic
2,0x91adf064659852bbfc67e4a597e4801b0ac653c24157...,1624916402,D,100,matic
3,0x928ff6a013f48a01d59228fe8538e39c28f66357f2ce...,1624916646,D,100,matic
4,0x0dc86e24b8dcb359e2c9ff484f6f020d429c8eaa66c2...,1624962834,D,100,matic
...,...,...,...,...,...
10,0x551ffe0e8a314c362167d4e33514628937fe0de52f0c...,1644249862,W,100000,matic
11,0x081eabfe3e3fa309b0629d7bc6d1f1ec00a242262df1...,1644307979,W,100000,matic
12,0x19c52ae5ac2a6816fb140e25b208e5bcf75f6b4ea1f0...,1644340425,W,100000,matic
13,0xf7006f5b652c4935efd0d9c841c109ee0d6763c5dbe5...,1647250450,W,100000,matic


data grouping

In [26]:
poolEvents['date']= pd.to_datetime(poolEvents['blockTimestamp'], unit='s').dt.date

In [30]:
dailyPoolEventsList = []
for date, df in poolEvents.groupby('date'):
    df['type'] = df['currency'] +'-'+ df['amount'] +'-'+ df['type']
    dayEvents = df.groupby('type')['id'].count().rename(date)
    dailyPoolEventsList.append(dayEvents)
dailyPoolEvents = pd.concat(dailyPoolEventsList, axis=1).sort_values(by='type').fillna(0).transpose()
cumPoolEvents = dailyPoolEvents.cumsum()
cumPoolEvents.to_json('daily-events-matic.json')

In [31]:
import pandas as pd
eventDf = pd.read_json('daily-events-matic.json')